In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
file_path = "data/overlap_category-V0.2.csv"
df = pd.read_csv(file_path, index_col=0)


In [59]:
overlap_matrix = df.to_numpy()

In [60]:
# Exclude the last column before summing
numeric_data = overlap_matrix[:, :-1]

# Convert the numeric part to integers (if necessary)
numeric_data = numeric_data.astype(int)

# Now sum along each row
S = np.sum(numeric_data, axis=1)


In [61]:
df_int = df[df["category"] == 'Internet Bank']

# Select only numeric columns
numeric_df = df_int.select_dtypes(include=[np.number])

# Calculate the sum across numeric columns for each row
df_int['Sum'] = numeric_df.sum(axis=1)

# Find the maximum sum value
max_sum = df_int['Sum'].max()
print("Maximum Sum:", max_sum)

Maximum Sum: 273


/tmp/ipykernel_5624/3714142479.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_int['Sum'] = numeric_df.sum(axis=1)


### The DataFrame is assumed to have:
####   - A "category" column (e.g., 'Internet Bank', 'Mega Bank', 'Hamrah Bank')
####   - Other columns (with numeric values) representing overlap scores among keywords.
####   - The index of the DataFrame holds the keyword names.


In [62]:
import pandas as pd

def compute_search_weights_by_group(df, mega_value, hamrah_value, internet_value):
    groups = {
        "Mega Bank": mega_value,
        "Hamrah Bank": hamrah_value,
        "Internet Bank": internet_value
    }
    
    result_dfs = []
    
    for group_name, user_value in groups.items():
        group_df = df[df["category"] == group_name].copy()
        
        group_keywords = group_df.index.tolist()
        
        group_overlap = group_df.loc[:, group_keywords]
       
        S_group = group_overlap.sum(axis=1)
        #  alpha is name of maximum overlap in each category
        alpha_index = S_group.idxmax()
        print("0:",alpha_index)
        alpha_keyword = S_group.loc[alpha_index]
        print("1:", alpha_keyword)
        alpha_keyword = group_keywords[group_keywords.index(alpha_index)]
        print("2:",alpha_keyword)
        

        overlaps_with_alpha = group_overlap[alpha_keyword]
        
        max_overlap = 1 if overlaps_with_alpha.max() ==0  else overlaps_with_alpha.max()

        print("overlaps_with_alpha 1",overlaps_with_alpha)
        print("----------------------------------")
        overlaps_with_alpha =   (user_value - (overlaps_with_alpha / max_overlap)  * (user_value-1)).round()
        print("overlaps_with_alpha2  ",overlaps_with_alpha)
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

        
        # if max_overlap > 0:
        #     final_weights = user_value - (user_value - 1) * (overlaps_with_alpha / max_overlap)
        # else:
        #     final_weights = pd.Series([user_value] * len(overlaps_with_alpha), index=overlaps_with_alpha.index)
     
        # # final_weights = final_weights.round().astype(int).clip(1, user_value)
        # final_weights = final_weights.round().astype(int).clip(1, user_value)
      
        # df_group_result = pd.DataFrame({
        #     "Keyword": group_keywords,
        #     "Raw Sum": S_group.values,
        #     "Alpha Keyword": alpha_keyword,  
        #     "Overlap with Alpha": overlaps_with_alpha.values,
        #     "Final Search Weight": final_weights.values,
        #     "Category": group_name
        # })
        
        # result_dfs.append(df_group_result)
    
    df_final = pd.concat(result_dfs, ignore_index=True)
  
    df_final = df_final.sort_values(by="Final Search Weight", ascending=False)
    return df_final

df_final = compute_search_weights_by_group(df, mega_value=8, hamrah_value=5, internet_value=10)

# display(df_final)


0: مگابانک همراه
1: 156
2: مگابانک همراه
overlaps_with_alpha 1 دانلود مستقیم بانک پلاس            5
دانلود مگا بانک ملت برای ایفون    19
دانلود مگابانک                    25
مگا بانک ملت نسخه وب              20
مگابانک                           26
مگابانک آخرین نسخه                18
مگابانک ملت                       24
مگابانک نسخه تحت وب               17
مگابانک همراه                      0
همراه پلاس ملت                     2
Name: مگابانک همراه, dtype: int64
----------------------------------
overlaps_with_alpha2   دانلود مستقیم بانک پلاس           7.0
دانلود مگا بانک ملت برای ایفون    3.0
دانلود مگابانک                    1.0
مگا بانک ملت نسخه وب              3.0
مگابانک                           1.0
مگابانک آخرین نسخه                3.0
مگابانک ملت                       2.0
مگابانک نسخه تحت وب               3.0
مگابانک همراه                     8.0
همراه پلاس ملت                    7.0
Name: مگابانک همراه, dtype: float64
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
0: دانلود همراه با

ValueError: No objects to concatenate